In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
df = pd.read_csv(r'processed_data2.csv')

In [2]:
df.sort_values(by=['optionid','date'], inplace=True)

In [95]:
df['optionid_chg'] = df['optionid'].diff(periods=-1) 
df['impvol_chg'] = df['impl_volatility'].diff(periods=-1) * 100
df['ret_chg'] = df['Index_Spot'].pct_change(periods=-1)

In [96]:
df.size

31894768

In [97]:
data = df[df["optionid_chg"]==0]
data.head()

,Unnamed: 0,Unnamed: 0.1,optionid,date,exdate,strike_price,best_bid,best_offer,impl_volatility,delta,days_to_expiry,time_to_maturity,Index_Spot,optionid_chg,impvol_chg,ret_chg
30,30,2386060,33033967.0,2010-01-04,2010-12-18,700000,422.3,426.3,0.331839,0.927833,348,0.953425,1132.99,0.0,0.3542,-0.003106
21,21,2387944,33033967.0,2010-01-05,2010-12-18,700000,424.9,428.9,0.328297,0.930406,347,0.950685,1136.52,0.0,0.1979,-0.000545
15,15,2389832,33033967.0,2010-01-06,2010-12-18,700000,425.0,429.0,0.326318,0.931496,346,0.947945,1137.14,0.0,-0.3351,-0.003985
76,76,2391720,33033967.0,2010-01-07,2010-12-18,700000,429.6,433.6,0.329669,0.931657,345,0.945205,1141.69,0.0,0.8194,-0.002873
91,91,2393585,33033967.0,2010-01-08,2010-12-18,700000,431.6,435.5,0.321475,0.935998,344,0.942466,1144.98,0.0,-0.3427,-0.001744


In [145]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.3)
train = data[:int(len(data)*0.7)]
test= data.drop(train.index)

In [146]:
def generate_Xy(data, reshape):
    X1 = data["ret_chg"]/np.sqrt(data["time_to_maturity"]);
    X2 = X1 * data["delta"]
    X3 = X1 * data["delta"] * data["delta"]
    X = np.array([X1,X2,X3])
    if reshape:
        X = X.reshape([len(X1),3])
    
    y = data["impvol_chg"]
    return X,y

In [147]:
X_train, y_train = generate_Xy(train, True)
X_test, y_test = generate_Xy(test, True)


In [148]:
Reg = LinearRegression(fit_intercept=False)
heston_model = Reg.fit(X_train, y_train)
coef = heston_model.coef_
score = heston_model.score(X_train,y_train)
print( "coef: ", coef )
print( "score: ", score )

coef:  [ 0.07573041  0.02941734 -0.02501317]
score:  -0.0024138730415819154


In [149]:
y_pred = heston_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(mse)

0.812543069159875


In [150]:
import statsmodels.api as sm
result = sm.OLS(y_train, X_train).fit()
print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:             impvol_chg   R-squared (uncentered):                   0.000
Model:                            OLS   Adj. R-squared (uncentered):             -0.000
Method:                 Least Squares   F-statistic:                             0.7037
Date:                Sat, 27 Nov 2021   Prob (F-statistic):                       0.550
Time:                        17:35:10   Log-Likelihood:                     -2.1191e+06
No. Observations:             1359638   AIC:                                  4.238e+06
Df Residuals:                 1359635   BIC:                                  4.238e+06
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [154]:
data_set = np.array([data['ret_chg'],data["time_to_maturity"],data["delta"],data['impvol_chg']])
dim_x, dim_y = data_set.shape
data_set = data_set.reshape(dim_y, dim_x)

In [ ]:
# pytorch mlp for binary classification

# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, df):
        self.X = df[:,:-1]
        self.y = df[:,-1]

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        self.hidden1 = nn.Linear(n_inputs, 10)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(10, 8)
        self.act2 = nn.ReLU()
        self.hidden3 = Linear(8, 1)
        self.act3 = Sigmoid()

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

# prepare the dataset
def prepare_data(df):
    # load the dataset
    dataset = CSVDataset(df)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=1024, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024*2, shuffle=False)
    return train_dl, test_dl

# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(10):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
            if( i%50 == 0 ):
                print(loss.data.item())

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

# prepare the data
#path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
train_dl, test_dl = prepare_data(data_set)
print(len(train_dl.dataset), len(test_dl.dataset))

In [183]:
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
import torch.nn as nn
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [180]:
# pytorch mlp for binary classification

# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, df):
        # load the csv file as a dataframe
        #df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df[:,:-1]
        self.y = df[:,-1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

# prepare the dataset
def prepare_data(df):
    # load the dataset
    dataset = CSVDataset(df)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=1024, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024*2, shuffle=False)
    return train_dl, test_dl

# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(10):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
            if( i%50 == 0 ):
                print(loss.data.item())

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

# prepare the data
#path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
train_dl, test_dl = prepare_data(data_set)
print(len(train_dl.dataset), len(test_dl.dataset))


1301368 640973


In [181]:
640973/len(test_dl)

2047.8370607028753

In [182]:
# define the network
model = MLP(3)
# train the model
train_model(train_dl, model)
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)
# make a single prediction (expect class=1)
#row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]


11423.9189453125
-16847214.0
-16169500.0
-17007368.0
-16552789.0
-15686357.0
-17142342.0
-17155090.0
-16974560.0
-16070769.0
-16615280.0
-17798986.0
-16687104.0
-17208956.0
-16725684.0
-16342250.0
-16464888.0
-16925782.0
-16850008.0
-15791834.0
-17184020.0
-15870552.0
-16148444.0
-16931908.0
-17200798.0
-16816460.0
-16769114.0
-16410886.0
-16626166.0
-17039988.0
-16417505.0
-17243642.0
-16807790.0
-16589477.0
-17077648.0
-16663139.0
-16863522.0
-16303840.0
-17104368.0
-16514328.0
-17189004.0
-16679528.0
-16056410.0
-16870116.0
-16368366.0
-16916710.0
-16468367.0
-16607684.0
-16776772.0
-16599369.0
-16173479.0
-17035430.0
-15995398.0
-16569506.0
-16596309.0
-16748730.0
-16957748.0
-17244350.0
-16730371.0
-16084205.0
-16731268.0
-16162152.0
-16394034.0
-17344100.0
-16321757.0
-16508630.0
-16834748.0
-16470039.0
-16153776.0
-16838646.0
-17354556.0
-16610435.0
-17042362.0
-16478731.0
-16453244.0
-16754570.0
-15859600.0
-16350082.0
-16015367.0
-16561790.0
-17053204.0
-17711452.0
-16346984.0

In [ ]:
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

In [ ]:
# ANN model training
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

for epoch in range(num_epochs):
    #for i, (images, labels) in enumerate(train_loader):
    for i, (train_inputs, train_targets) in enumerate(train_dl):

        optimizer.zero_grad() # Clear gradients
        outputs = model(train_inputs) # Forward propagation
        loss = error(outputs, train_targets) # Calculate softmax and cross entropy loss
        loss.backward() # Calculating gradients
        optimizer.step() # Update parameters
        
        count += 1
        
        if count % 50 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            
            # Predict test dataset
            for i, (test_inputs, test_targets) in enumerate(test_dl):
            for images, labels in test_loader:
                test = Variable(images.view(-1, 28*28))
                outputs = model(test_inputs) # Forward propagation
                predicted = outputs.detach().numpy()
                total += len(labels) # Total number of labels
                correct += (predicted == labels).sum() # Total correct predictions
            
            accuracy = 100.0 * correct.item() / total
            
            # store loss and iteration
            loss_list.append(loss.data.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 500 == 0:
                print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data.item(), accuracy))